In [10]:
import os
import cv2
import tqdm
import numpy as np
from xml.etree import ElementTree

In [40]:
def resize_dataset(image, new_heigh, save_dir):  

    assert isinstance(new_heigh, int)
    
    basename_img = os.path.basename(image)
    base = basename_img.split('.')[0]
    root = os.path.dirname(image)    
    xml_name = os.path.join(root, base+'.xml')    
    assert os.path.exists(xml_name)
    
    img = cv2.imread(image) 
    assert isinstance(img, np.ndarray)
    
    width = img.shape[1]
    height = img.shape[0]    
    n = height//new_heigh   
    # check input heigth to correct size
    assert width%n == 0 and height%n == 0
    
    # resize image
    new_width = width//n
    new_height = height//n
    new_size = (new_width, new_height)
    new_img = cv2.resize(img, new_size)
    save_path = os.path.join(save_dir, basename_img)    
    cv2.imwrite(save_path, new_img)    

    # resize xml
    basename_xml = os.path.basename(xml_name)
    xml_save_name = os.path.join(save_dir, basename_xml)    
    tree = ElementTree.parse(xml_name)
    root = tree.getroot()
    for i in root:      
        if i.tag == 'object':
            for el in i:                  
                if el.tag == 'polygon':                   
                    for j in el:                       
                        if j.tag == "pt":                       
                            for c in j:                                  
                                if c.tag == 'x' or c.tag == 'y': 
                                    new_coord = float(c.text)/n
                                    new_coord = round(new_coord,2)           
                                    c.text = str(new_coord)                                
    tree.write(xml_save_name)     

In [38]:
origin_dir = '/home/alex/poker/datasets_for_detector_train/gg_2400'
resolutions = ['jpg', 'webp', 'jpeg']
imgs = [os.path.join(origin_dir,i) for i in os.listdir(origin_dir) \
        if i.split('.')[-1] in resolutions]
len(imgs)

53

In [18]:
test_n = 1200
base = os.path.basename(origin_dir)
root = os.path.dirname(origin_dir)
new_folder = os.path.join(root, base+'_resize_'+str(test_n))
if not os.path.exists(new_folder):
    os.mkdir(new_folder)

In [41]:
for i in tqdm.tqdm_notebook(imgs):
    resize_dataset(i, test_n, new_folder)
    

In [43]:
base = os.path.basename(imgs[0])
example = os.path.join(new_folder, base)
print (example)
example = cv2.imread(example)
example.shape

/home/alex/poker/datasets_for_detector_train/gg_2400_resize_1200/image0000000105.jpg


(1200, 540, 3)